In [ ]:
import streamlit as st
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq  
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

import os 

# doubts in [create_history_aware_retriever, MessagesPlaceholder, RunnableWithMessageHistory]
# how chat history stored



from dotenv import load_dotenv

In [34]:
from dotenv import load_dotenv
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

## LLM model
llm = ChatGroq(model="gemma2-9b-it")

## Embeddings: 2304
embeddings = OllamaEmbeddings(model="gemma2:2b")

loader = PyPDFLoader(r"temp.pdf")
docs = loader.load()

# Split the documents and create embeddings
textSplitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap = 200)
splittedDocs=textSplitter.split_documents(docs)

vectorStore = FAISS.from_documents(documents=splittedDocs, embedding= embeddings)
retrieverDB  = vectorStore.as_retriever()

In [32]:
arr=vectorStore._embed_query("hwllo how are you")


In [12]:
len(arr)

2304

In [ ]:
contextualize_q_system_prompt = (
        """ 
            Given a chat history and the latest user question. 
            which might reference context in the chat history 
            Formulate a standalone question which can be understood. 
            without the chat history, Do not answer the question. Just formulate it if needed
            otherwise return it  as it is. 
        """
    )

contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system",contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human","{input}")
        ]
    )
# it's task is to take the user input, then rephrase it by llm using history,
# Then rephrase question is passed to vectorRetriver for getting documents context

history_awareRetriever=create_history_aware_retriever(llm, retrieverDB, contextualize_q_prompt)


In [14]:
history_awareRetriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B07B5A0350>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=' \n            Given a chat history and the latest user question. \n            which might reference context in the chat history \n            Formulate a standalone question which can be understood. \n            without th

In [29]:
system_prompt = (
        """ 
            You are the assistant for question answering tasks. 
            Use the following pieces of retriever context to answer the question.
            If you don't know the answer, Say thank you don't know. Use the three sentences maximum
            and keep the answer concise.
            \n\n
            {context}.
        """
    )
qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system",system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human","{input}")
        ]
    )
# context fetch by histrory aware retriever is passed to create_stuff_documents_chain
# and it is responsible for giving "context" to system prompt or LLM. & generate the response. 

que_ans_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_awareRetriever, que_ans_chain)


In [16]:
que_ans_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template=" \n            You are the assistant for question answering tasks. \n            Use the following pieces of retriever context to answer the question.\n            If you don't know the answer, Say thank you don't know. Use the three sentences maximum\n            and keep the answer concise.\n            \n\n\n            {context}.\n        ")), Messages

In [17]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B07B5A0350>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=' \n            Given a chat history and the latest user question. \n            which might reference context in the chat history \n            Formulate a s

In [21]:
# history for session

store= {}
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [22]:
conversational_rag_chain = RunnableWithMessageHistory(
            rag_chain, get_session_history, input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer" # optional
    )   



In [23]:


user_input = "what is saving"

response = conversational_rag_chain.invoke(
            {"input":user_input},
            config = {
                "configurable":{"session_id":"subodh1"}
            }
        )

In [24]:

response["answer"]


'Saving involves setting aside money for future use, typically focusing on short-term needs and preserving capital.  \n'

In [25]:
user_input = "what question i asked you before"

response = conversational_rag_chain.invoke(
            {"input":user_input},
            config = {
                "configurable":{"session_id":"subodh1"}
            }
        )

In [26]:

response["answer"]

'You asked: "what is saving" \n'